<a href="https://colab.research.google.com/github/manthanthakker/AI/blob/master/AudioClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

In [2]:
%%capture
!pip install datasets==1.14
!pip install transformers==4.11.3
!pip install librosa

In [42]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [4]:
%%capture
!apt install git-lfs

# Step 1: Dataset 

In [5]:
from datasets import load_dataset, load_metric

In [122]:
dataset = load_dataset("superb", "ks")
metric = load_metric("accuracy")

Reusing dataset superb (/root/.cache/huggingface/datasets/superb/ks/1.9.0/ce836692657f82230c16b3bbcb93eaacdbfd7de4def3be90016f112d68683481)


  0%|          | 0/3 [00:00<?, ?it/s]

In [123]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 51094
    })
    validation: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 6798
    })
    test: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 3081
    })
})

In [128]:
dataset["train"][1]

{'audio': {'array': array([ 0.        ,  0.        ,  0.        , ...,  0.00289917,
          0.00338745, -0.00033569], dtype=float32),
  'path': '/root/.cache/huggingface/datasets/downloads/extracted/05734a36d88019a09725c20cc024e1c4e7982e37d7d55c0c1ca1742ea1cdd47f/_background_noise_/dude_miaowing.wav',
  'sampling_rate': 16000},
 'file': '/root/.cache/huggingface/datasets/downloads/extracted/05734a36d88019a09725c20cc024e1c4e7982e37d7d55c0c1ca1742ea1cdd47f/_background_noise_/dude_miaowing.wav',
 'label': 10}

In [34]:
!git clone https://github.com/Nikunj1729/free-spoken-gujarati-digit-dataset

Cloning into 'free-spoken-gujarati-digit-dataset'...
remote: Enumerating objects: 3333, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 3333 (delta 10), reused 0 (delta 0), pack-reused 3312
Receiving objects: 100% (3333/3333), 281.99 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (1316/1316), done.
Checking out files: 100% (1956/1956), done.


In [170]:
from datasets import load_dataset, load_metric
train_data=load_dataset("/content/load_guj/load_guj.py",split="train")
val_data=load_dataset("/content/load_guj/load_guj.py",split="test")
test_data=val_data

No config specified, defaulting to: new_dataset/train


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

No config specified, defaulting to: new_dataset/train
Reusing dataset new_dataset (/root/.cache/huggingface/datasets/new_dataset/train/1.1.0/5c4ed9612e3ddae614ebb7d02e2394d863273f0c537da95ce3583516b4863932)


Dataset new_dataset downloaded and prepared to /root/.cache/huggingface/datasets/new_dataset/train/1.1.0/5c4ed9612e3ddae614ebb7d02e2394d863273f0c537da95ce3583516b4863932. Subsequent calls will reuse this data.


In [139]:
test_data

Dataset({
    features: ['file', 'audio', 'label'],
    num_rows: 388
})

In [138]:
!ls /content/

load_guj  sample_data  test.jsonl  train.jsonl	wav2vec2-base-finetuned-ks


In [86]:
import glob
all_wavs=glob.glob("/content/load_guj/**/*.wav", recursive=True)
len(all_wavs)
from sklearn.model_selection import train_test_split
x, Y=train_test_split(all_wavs, test_size=0.2)
print(len(x))
print(len(Y))
import json
#train_file = open("/content/load_guj/train.jsonl", "w")
#n = train_file.write(json.dumps(x))
#train_file.close()

import json

with open('/content/load_guj/train.jsonl', 'w') as outfile:
    for entry in x:
        json.dump(entry, outfile)
        outfile.write('\n')

with open("/content/load_guj/test.jsonl", "w") as outfile:
    for entry in Y:
        json.dump(entry, outfile)
        outfile.write('\n')

1552
388


In [168]:
from datasets.dataset_dict import DatasetDict

In [9]:
dataset["train"]

Dataset({
    features: ['file', 'audio', 'label'],
    num_rows: 51094
})

In [10]:
dataset["train"][1]["audio"]

{'array': array([ 0.        ,  0.        ,  0.        , ...,  0.00289917,
         0.00338745, -0.00033569], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/05734a36d88019a09725c20cc024e1c4e7982e37d7d55c0c1ca1742ea1cdd47f/_background_noise_/dude_miaowing.wav',
 'sampling_rate': 16000}

In [11]:
dataset["train"][1]["label"]

10

In [12]:
 dataset["train"].features["label"]

ClassLabel(num_classes=12, names=['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', '_silence_', '_unknown_'], names_file=None, id=None)

In [13]:
dataset["train"]

Dataset({
    features: ['file', 'audio', 'label'],
    num_rows: 51094
})

In [14]:
dataset["train"][2]

{'audio': {'array': array([ 0.        ,  0.        ,  0.        , ..., -0.02764893,
         -0.01760864,  0.00912476], dtype=float32),
  'path': '/root/.cache/huggingface/datasets/downloads/extracted/05734a36d88019a09725c20cc024e1c4e7982e37d7d55c0c1ca1742ea1cdd47f/_background_noise_/exercise_bike.wav',
  'sampling_rate': 16000},
 'file': '/root/.cache/huggingface/datasets/downloads/extracted/05734a36d88019a09725c20cc024e1c4e7982e37d7d55c0c1ca1742ea1cdd47f/_background_noise_/exercise_bike.wav',
 'label': 10}

In [171]:
labels = ["0","1","2","3","4","5","6","7","8","9"]
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["8"]

'8'

In [172]:
dataset={}
dataset["train"]=train_data
dataset["test"]=test_data

In [173]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset["train"])-1)
    example = dataset["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["label"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: 0
Shape: (10817,), sampling rate: 16000



Label: 7
Shape: (12275,), sampling rate: 16000



Label: 5
Shape: (12035,), sampling rate: 16000



Label: 2
Shape: (13340,), sampling rate: 16000



Label: 7
Shape: (12811,), sampling rate: 16000


# Step 2: Pre processing

In [174]:
max_duration = 1.0  # seconds

In [175]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
 

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [176]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [179]:
preprocess_function(dataset['train'][:5])

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


{'input_values': [array([ 0.00129049, -0.00974802, -0.0088512 , ..., -0.02515105,
       -0.01479916,  0.00070186], dtype=float32), array([ 0.00026717, -0.00900791, -0.0188656 , ...,  0.01414172,
        0.00430038, -0.00017345], dtype=float32), array([-6.1017275e-03, -4.5385023e-04,  7.0025874e-03, ...,
        1.6004423e-02,  8.2106097e-03,  2.9551686e-05], dtype=float32), array([ 0.05729007,  0.08500836,  0.05893154, ..., -0.02786119,
       -0.02472607,  0.00191057], dtype=float32), array([-2.9205489e-03, -3.5297568e-03, -2.5042519e-03, ...,
        1.7325737e-03, -7.7132587e-03, -9.6490672e-05], dtype=float32)]}

In [178]:
dataset=DatasetDict(dataset)

In [180]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio", "file"], batched=True)
encoded_dataset

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_values', 'label'],
        num_rows: 1552
    })
    test: Dataset({
        features: ['input_values', 'label'],
        num_rows: 388
    })
})

# Gujarati Dataset

# Training/Finetuning

In [182]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
 

In [183]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    warmup_ratio=0.1,
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [184]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [185]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

/content/wav2vec2-base-finetuned-ks is already a clone of https://huggingface.co/manthan40/wav2vec2-base-finetuned-ks. Make sure you pull the latest changes with `repo.git_pull()`.


In [186]:
trainer.train()

***** Running training *****
  Num examples = 1552
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 12


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [188]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-base-finetuned-ks
Configuration saved in wav2vec2-base-finetuned-ks/config.json
Model weights saved in wav2vec2-base-finetuned-ks/pytorch_model.bin
Configuration saved in wav2vec2-base-finetuned-ks/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 3.34k/361M [00:00<?, ?B/s]

Upload file runs/May12_02-15-52_f6718c83993a/events.out.tfevents.1652321761.f6718c83993a.63.4:  63%|######3   …

Upload file training_args.bin: 100%|##########| 2.80k/2.80k [00:00<?, ?B/s]

Upload file runs/May12_02-15-52_f6718c83993a/1652321761.5831435/events.out.tfevents.1652321761.f6718c83993a.63…

Upload file runs/May12_02-07-11_f6718c83993a/1652321255.5379248/events.out.tfevents.1652321255.f6718c83993a.63…

Upload file runs/May12_02-07-11_f6718c83993a/events.out.tfevents.1652321255.f6718c83993a.63.2:  72%|#######1  …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/manthan40/wav2vec2-base-finetuned-ks
   e21180a..03263af  main -> main

Dropping the following result as it does not have all the necessary field:
{'dataset': {'name': 'new_dataset', 'type': 'new_dataset', 'args': 'train'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/manthan40/wav2vec2-base-finetuned-ks
   03263af..ddd9315  main -> main



'https://huggingface.co/manthan40/wav2vec2-base-finetuned-ks/commit/03263af18870eda731ea369900c896c72c826da5'

In [187]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 388
  Batch size = 32


Epoch,Training Loss,Validation Loss


IndexError: ignored